In [4]:
import configparser

def get_key(app):
    config = configparser.ConfigParser()
    config.read('../../key/config.ini')
    api_key = config.get(app, 'api_key')
    return api_key

def get_gaode_key():
    return get_key("GaoDe")

def get_openai_key():
    return get_key("OpenAI")

In [5]:
import openai
import json
import requests
openai.api_key = get_openai_key()


#调用GPT模型
def get_completion_from_messages(messages,functions, model="gpt-3.5-turbo-0613", temperature=0):
    print("-----------------------------------------\n输入messages:"+str(json.dumps(messages, indent=1, ensure_ascii=False)))
    print("输入functions:"+str(json.dumps(functions, indent=1, ensure_ascii=False)))
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        functions=functions
    )
    
    print("输出:"+json.dumps(response, indent=1, ensure_ascii=False))
    return response.choices[0]


def get_current_weather(para):
    city = para["city"]
    url = f'https://restapi.amap.com/v3/weather/weatherInfo?parameters'
    params = {
        'key': get_gaode_key(),
        'city': city,
        'extensions': 'base',
        'output': 'JSON',
        'extensions':'all'
        
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    return data['forecasts'][0]['casts'][0]

def send_email(para):
    addr = para["addr"]
    title = para["title"]
    content = para["content"]
    text = f"To {addr}\n{title}\n{content}"
    #print("邮件已经发送：\n "+text)
    return {"status":"邮件已经发送","email":text}

functions = [
    {
        "name": "get_current_weather",
        "description": "获取一个城市的当前天气",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "城市名称"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "send_email",
        "description": "给指定地址发送电子邮件",
        "parameters": {
            "type": "object",
            "properties": {
                "addr": {
                    "type": "string",
                    "description": "电子邮件地址"
                },
                "title": {
                    "type": "string",
                    "description": "邮件标题"
                },
                "content": {
                    "type": "string",
                    "description": "邮件内容"
                }
            },
            "required": ["addr","title","content"]
        }
    }
]


#向GPT提问
def get_completion(prompt,functions):
    #拼接消息
    messages = [{"role": "user", "content": prompt}]
    
    #第一轮调用GPT
    response = get_completion_from_messages(messages,functions)
    
    #如果返回终止原因是API调用，本地执行
    if response['finish_reason'] == "function_call":   
        arguments = response["message"]["function_call"]["arguments"]
        function =  response["message"]["function_call"]["name"]

        result = eval(f"{function}({arguments})")#根据API名称和参数动态调用本地接口
        
        #把GPT的调用指令和API返回结果再次发送给GPT
        messages.append(response["message"])
        messages.append( {"role": "function", "name": "get_current_weather", "content":str(result)})
        response = get_completion_from_messages(messages,functions)
        
    print(response["message"]["content"])
    

get_completion("我在杭州,今天出门要带伞吗",functions)

-----------------------------------------
输入messages:[
 {
  "role": "user",
  "content": "我在杭州,今天出门要带伞吗"
 }
]
输入functions:[
 {
  "name": "get_current_weather",
  "description": "获取一个城市的当前天气",
  "parameters": {
   "type": "object",
   "properties": {
    "city": {
     "type": "string",
     "description": "城市名称"
    }
   },
   "required": [
    "city"
   ]
  }
 },
 {
  "name": "send_email",
  "description": "给指定地址发送电子邮件",
  "parameters": {
   "type": "object",
   "properties": {
    "addr": {
     "type": "string",
     "description": "电子邮件地址"
    },
    "title": {
     "type": "string",
     "description": "邮件标题"
    },
    "content": {
     "type": "string",
     "description": "邮件内容"
    }
   },
   "required": [
    "addr",
    "title",
    "content"
   ]
  }
 }
]
输出:{
 "id": "chatcmpl-7STNJiY2P0KjdNFAoxv2fw0cg5jZP",
 "object": "chat.completion",
 "created": 1687020457,
 "model": "gpt-3.5-turbo-0613",
 "choices": [
  {
   "index": 0,
   "message": {
    "role": "assistant",
    "con